# Query files & datasets

We saw how LaminDB allows to query & search across files & datasets using registries: {doc}`meta`.

This guide addresses "queries within datasets", for instance:
```
df = ln.File.filter(labels__name == "setosa").first().load()  # access a batch of the iris dataset ingested in the tutorial
df_setosa = df.loc[:, df.iris_species_name == "setosa"]  # subset the iris dataset to observations of species "setosa"
```

Because the file was validated, subsetting the `DataFrame` is guaranteed to succeed, sparing you the headache of re-curating features & labels.

Such within-dataset queries are also possible for cloud-backed datasets using DuckDB, TileDB, zarr, HDF5, parquet and other storage backends.

While LaminDB doesn't offer a direct query API for these datasets, it allows accessing the object's APIs through an accessor: {meth}`~lamindb.File.backed`.

Here, we show how to subset an `AnnData` and generic `HDF5` and `zarr` datasets accessed in the cloud.

## Setup

In [ ]:
!lamin init --storage s3://lamindb-ci --name test-data

In [ ]:
import lamindb as ln

In [ ]:
ln.settings.verbosity = "info"

We'll need some test data:

In [ ]:
ln.File("s3://lamindb-ci/lndb-storage/pbmc68k.h5ad").save()
ln.File("s3://lamindb-ci/lndb-storage/testfile.hdf5").save()

## AnnData

An `h5ad` file stored on s3:

In [ ]:
file = ln.File.filter(key="lndb-storage/pbmc68k.h5ad").one()

In [ ]:
file.path

In [ ]:
adata = file.backed()

This object is an `AnnDataAccessor` object, an `AnnData` object backed in the cloud:

In [ ]:
adata

Without subsetting, the `AnnDataAccessor` object references underlying lazy `h5` or `zarr` arrays:

In [ ]:
adata.X

You can subset it like a normal `AnnData` object:

In [ ]:
obs_idx = adata.obs.cell_type.isin(["Dendritic cells", "CD14+ Monocytes"]) & (
    adata.obs.percent_mito <= 0.05
)
adata_subset = adata[obs_idx]
adata_subset

Subsets load arrays into memory upon direct access:

In [ ]:
adata_subset.X

To load the entire subset into memory as an actual `AnnData` object, use `to_memory()`:

In [ ]:
adata_subset.to_memory()

## Generic HDF5

Let us query a generic HDF5 file:

In [ ]:
file = ln.File.filter(key="lndb-storage/testfile.hdf5").one()

And get a backed accessor:

In [ ]:
backed = file.backed()

The returned object contains the `.connection` and `h5py.File` or `zarr.Group` in `.storage`

In [ ]:
backed

In [ ]:
backed.storage

In [ ]:
!lamin delete --force test-data